<a href="https://colab.research.google.com/github/3C-Computing-WA3-Project/WA3/blob/feature%2Fzhihao%2Ffinalize-quiz-system/WA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global PIP Installation

In [45]:
%pip install beanie bcrypt traitlets --q

# Global Importaion

In [46]:
import asyncio
import hashlib
import inspect
import math
import nest_asyncio
import time
import types
import random
from datetime import datetime

from beanie import init_beanie, Document, Indexed, PydanticObjectId
from IPython.display import clear_output, display
from motor.motor_asyncio import AsyncIOMotorClient
from google.colab import userdata
from pydantic import BaseModel, Field
from pymongo.errors import DuplicateKeyError
from traitlets import HasTraits, Unicode, Instance, dlink
from typing import Optional, final
import bcrypt
import ipywidgets as widgets

loop = asyncio.get_event_loop()
nest_asyncio.apply()

# Simplified MVC Framework

In [47]:
class AppState(HasTraits):
    '''
    Dynamic app data
    '''
    userId = Instance(PydanticObjectId)
    name = Unicode()

class WidgetTemplate():
    directory = dict() # key: prototype instance, value: resultant WidgetTemplate instance id
    def __init__(self, prototype: widgets.Widget):
        self._cls = prototype.__class__
        self._state = {key:value
                       for key, value in prototype.get_state(drop_defaults=True).items()
                       if key.startswith("_")}

        self._args = {key:value
                       for key, value in prototype.get_state(drop_defaults=True).items()
                       if not key.startswith("_")
                       and key not in ["layout", "style", "children"]}
        '''
        For widgets.Select special case
        '''
        _options = prototype.get_state(drop_defaults=True).get("_options_labels")
        if _options:
            self._args["options"] = list(_options)

        self._children = list(self.__class__.directory[x] for x in getattr(prototype, "children", tuple()))

        self._layout = {key:value
                        for key, value in prototype.layout.trait_values().items()
                        if not key.startswith("_")
                        and key not in ["comm", "keys"]}

        self._style = None
        if not isinstance(prototype, widgets.Box):
            self._style = { key:value
                            for key, value in prototype.style.trait_values().items()
                            if not key.startswith("_")
                            and key not in ["comm", "keys"]}

        self._extra_attrs = {key:value
                             for key, value in vars(prototype).items()
                             if not key.startswith("_")
                             }

        self.__class__.directory[prototype] = id(self)

    def build(self, resultant_instances):
        clone = self._cls(**self._args)
        clone.set_state(self._state)
        children = list()
        for x in self._children:
            children.append(resultant_instances[x])

        if len(children):
            clone.children = children

        clone.layout = widgets.Layout(**self._layout)
        if self._style is not None:
            clone.style = type(clone.style)(**self._style)

        for key, value in self._extra_attrs.items():
            setattr(clone, key, value)

        return clone

class ViewBase():
    def __init_subclass__(cls):
        cls.template = dict()
        for name, val in vars(cls).items():
            if isinstance(val, widgets.Widget):
                cls.template[name] = WidgetTemplate(val)
    @final
    def __init__(self, appstate):
        self.resultant_instances = dict() # key: WidgetTemplate instance id, value: resultant widget instance
        self.appstate = appstate
        self.widgets_dict = dict()
        self.to_render_widgets = list()

        for name, val in self.__class__.template.items():
            widget = val.build(self.resultant_instances)
            self.resultant_instances[id(val)] = widget

            self.widgets_dict[name] = widget
            setattr(self, name, widget)

            if not getattr(widget, "isIgnored", False):
                self.to_render_widgets.append(widget)

        self.inject_before_init_sync()
        loop.run_until_complete(self.inject_before_init_async())

    @final
    def to_render(self) -> widgets.VBox:
        return widgets.VBox(
                self.to_render_widgets,
                layout = widgets.Layout(
                    max_width ="100%",
                    align_items="center"
                    )
                )

    @final
    def destroy_all(self):
        for widget in self.widgets_dict.values():
            widget.close()

    def inject_before_init_sync(self):
        pass

    async def inject_before_init_async(self):
        pass

    @final
    def binder(self, widget, nameoftrait, transform=None):
        '''
        A helper method that explicitly link the app state trait to the expected widget.
        '''
        if isinstance(widget, widgets.Widget) and hasattr(self.appstate, nameoftrait):
            if transform:
                dlink((self.appstate, nameoftrait), (widget, "value"), transform=transform)
            else:
                dlink((self.appstate, nameoftrait), (widget, "value"))
        else:
            raise TypeError(f"Expected Widget type and valid trait name. Got widget: {type(widget)}, name of trait: {nameoftrait}")

class ControllerBase():
    _required_arguments = {}

    def __init_subclass__(cls):
        if not hasattr(cls, "_obj_view"):
            raise AttributeError(f"{cls.__name__} must define _obj_view.")

        if not isinstance(cls._obj_view, type) or not issubclass(cls._obj_view, ViewBase):
            raise TypeError(f"{cls.__name__}._obj_view must be a subclass of ViewBase")

        if hasattr(cls, "_required_arguments"):
            if not isinstance(cls._required_arguments, dict):
                raise TypeError(f"{cls.__name__}._required_arguments must be a dict")

            if not all(isinstance(x, str) for x in cls._required_arguments.keys()):
                raise TypeError(f"{cls.__name__}._required_arguments must have string keys")

    @final
    def __init__(self, appstate, router, **kwargs):
        for key, expected_type in self.__class__._required_arguments.items():
            if key not in kwargs:
                raise ValueError(f"Missing required argument: '{key}'")
            if not (isinstance(kwargs[key], expected_type) or callable(kwargs[key])):
                raise TypeError(f"Argument '{key}' should be of type {expected_type.__name__}, got {type(kwargs[key]).__name__}")

            setattr(self, key, kwargs[key])

        self.kwargs = kwargs
        self.appstate = appstate
        self.router = router
        self._obj_view = self.__class__._obj_view

    @final
    def binding(self):
        for name, widget in self.view.widgets_dict.items():
            func = None

            if isinstance(widget, widgets.Button):
                func = getattr(self, f"on_{name}")
            else:
                continue

            if inspect.iscoroutinefunction(func):
                async def wrapper(event, f=func, w=widget):
                    w.disabled = True
                    await f(event)
                    w.disabled = False

                widget.on_click(lambda event: loop.run_until_complete(wrapper(event)))

            else:
                def wrapper(event, f=func, w=widget):
                    w.disabled = True
                    f(event)
                    w.disabled = False
                widget.on_click(wrapper)

    @final
    def show(self, isClear=True, isRebuild=False, isDisplay=True):
        if hasattr(self, "view") and isRebuild:
            self.view.destroy_all()

        if not hasattr(self, "view") or isRebuild:
            self.view = self._obj_view(self.appstate)
            self.binding()

        self.inject_before_show_sync()
        loop.run_until_complete(self.inject_before_show_async())

        if isDisplay:
            with self.router.container:
                display(self.view.to_render(), clear = isClear)

    @final
    def clone(self, n: int):
        for x in range(0, n):
            controller = type(self)(self.appstate, self.router, **self.kwargs)
            controller.show(isClear=False, isDisplay=False)
            yield controller.view.to_render()

    def inject_before_show_sync(self):
        pass

    async def inject_before_show_async(self):
        pass

class Router:
    def __init__(self):
        self.appstate = AppState()
        self.controllers = dict()
        self.container = widgets.Output()
        display(self.container)

    def register_one(self, controller: ControllerBase, **kwargs):
        '''
        As `AppState` class is an instance, I cannot pre-define the value in the ControllerBase and ViewBase class,
        so that I have to inject the AppState instance during runtime.
        '''
        if not issubclass(controller, ControllerBase):
            raise TypeError(f"Expected ControllerBase type got {type(controller)}")

        self.controllers[controller.__name__] = controller(self.appstate, self, **kwargs)

    def go(self, controller: ControllerBase, **kwargs):
        '''
        `controller` argument takes ControllerBase object.
        This function go the given `controller`
        '''
        self.controllers[controller.__name__].show(**kwargs)

# Data Models

In [48]:
class Quiz(BaseModel):
    question: str
    answer: list[str]
    solution: str

In [49]:
class UserModel(Document):
    name: str
    username: Indexed(str, unique=True)
    hashedPassword: str

    class Settings:
        collection = "UserModel"

In [50]:
class QuestionModel(Document):
    question_content: Quiz
    title: str
    question_hash: Indexed(str, unique=True)
    class Settings:
        collection = "QuestionsBank"

In [51]:
class AttemptModel(Document):
    userId: PydanticObjectId
    questionId: PydanticObjectId
    timeStamp: datetime = Field(default_factory=datetime.now)
    isCorrect: bool = False
    '''
    Every submission counts just before it is answered correctly.
    The default of `timesOfAnswer` is one as the model will only be inserted if the user submit the answer.
    '''
    timesOfAnswering: int = 1
    class Settings:
        collection = "AttemptModel"

# App

## Make Title

In [52]:
def make_title(text, _layout=None):
    return widgets.HTML(f"<h1 style='color: teal'>{text}</h1>",layout=_layout if _layout is not None else widgets.Layout(
        text_align="center"
))

## Main Menu UI

### Main Menu View

In [53]:
class MainMenuView(ViewBase):
    title = make_title("Main Menu")

    btn_login = widgets.Button(
            description="Login",
            disabled=False,
            button_style="primary"
    )

    btn_register = widgets.Button(
            description="Register",
            disabled=False,
            button_style="primary"
    )

    btn_exit = widgets.Button(
            description="Exit",
            disabled=False,
            button_style="danger"
    )

### Main Menu Controller

In [54]:
class MainMenuController(ControllerBase):
    _obj_view = MainMenuView

    def on_btn_login(self, event):
        self.router.go(LoginController, isRebuild=True)

    def on_btn_register(self, event):
        self.router.go(RegisterController, isRebuild=True)

    def on_btn_exit(self, event):
        pass

## Login UI

### Login View

In [55]:
class LoginView(ViewBase):
    title = make_title("Login Page")

    username = widgets.Text(
        placeholder='Enter your username',
        description='Username: ',
        disabled=False,
    )

    password = widgets.Password(
        description='Password:',
        disabled=False,
    )

    error_text = widgets.HTML(
        value="<strong style='color:red'>Invalid username or/and password</strong>",
        layout=widgets.Layout(display="none")
    )

    btn_exit = widgets.Button(
        description="Exit",
        disabled=False,
        button_style="danger"
    )
    btn_exit.isIgnored = True

    btn_login = widgets.Button(
        description="Login",
        disabled=False,
        button_style="primary"
    )
    btn_login.isIgnored = True

    box = widgets.HBox([btn_exit, btn_login], layout=widgets.Layout(
        justify_content="space-between",
        grid_gap="1.5em"
    ))

### Login Controller

In [56]:
class LoginController(ControllerBase):
    _obj_view = LoginView

    def on_btn_exit(self, event):
        self.router.go(MainMenuController)

    async def on_btn_login(self, event):
        self.view.error_text.layout.display = "none"

        username = self.view.username.value
        pwd = self.view.password.value

        result = await UserModel.find_one(UserModel.username == username)
        hash = ""
        fakeHash = bcrypt.hashpw(b"invalid", bcrypt.gensalt()) # the register function restricted the lenght of password to be at least eight-character long. Suggested by https://cheatsheetseries.owasp.org/cheatsheets/Authentication_Cheat_Sheet.html#authentication-responses

        if result is not None:
            hash = result.hashedPassword.encode("utf-8")
        else:
            hash = fakeHash

        isAuth = bcrypt.checkpw(pwd.encode("utf-8"), hash) and result is not None
        if isAuth:
            self.appstate.name = result.name
            self.appstate.userId = result.id
            self.router.go(DashboardController)
        else:
            self.view.error_text.layout.display = ""

## Register UI

### Register View

In [57]:
class RegisterView(ViewBase):
    title = make_title("Register Page")

    name = widgets.Text(
            placeholder='Enter your name',
            description='Name: ',
            disabled=False,
            style={'description_width': '140px'}
    )

    username = widgets.Text(
        placeholder='Enter your username',
        description='Username: ',
        disabled=False,
        style={'description_width': '140px'}
    )

    password = widgets.Password(
        description='Password:',
        disabled=False,
        style={'description_width': '140px'}
    )

    confirmed_password = widgets.Password(
        description='Confirmed Password:',
        disabled=False,
        style={'description_width': '140px'}
    )

    error_text_password_not_match = widgets.HTML(
        value="<strong style='color:red'>Password does not match!</strong>",
        layout=widgets.Layout(display="none")
    )

    error_text_password_length = widgets.HTML(
        value="<strong style='color:red'>Password is too short!</strong>",
        layout=widgets.Layout(display="none")
    )

    error_text_username = widgets.HTML(
        value="<strong style='color:red'>This username is chosen!</strong>",
        layout=widgets.Layout(display="none")
    )

    btn_exit = widgets.Button(
        description="Exit",
        disabled=False,
        button_style="danger"
    )
    btn_exit.isIgnored = True

    btn_register = widgets.Button(
        description="Register",
        disabled=False,
        button_style="primary"
    )
    btn_register.isIgnored = True

    box = widgets.HBox([btn_exit, btn_register], layout=widgets.Layout(
        justify_content="space-between",
        grid_gap="1.5em"
    ))

    succeeded = widgets.HTML(
        value="<strong style='color:green'>Successfully registered an account! Click 'exit' to return back.</strong>",
        layout=widgets.Layout(display="none")
    )

### Register Controller

In [58]:
class RegisterController(ControllerBase):
    _obj_view = RegisterView

    def on_btn_exit(self, event):
        self.router.go(MainMenuController)

    async def on_btn_register(self, event):
        try:
            self.view.error_text_password_not_match.layout.display = "none"
            self.view.error_text_password_length.layout.display = "none"
            self.view.error_text_username.layout.display = "none"
            self.view.succeeded.layout.display = "none"

            pwd = self.view.password.value
            conf = self.view.confirmed_password.value

            # validation
            if pwd != conf:
                self.view.error_text_password_not_match.layout.display = ""
                return

            if len(pwd) < 8:
                self.view.error_text_password_length.layout.display = ""
                return

            salt = bcrypt.gensalt()
            hashed = bcrypt.hashpw(pwd.encode("utf-8"), salt)

            data = UserModel(
                name=self.view.name.value,
                username=self.view.username.value,
                hashedPassword=hashed
            )
            self.data = data
            await data.insert()
            self.view.succeeded.layout.display = ""

        except DuplicateKeyError:
            self.view.error_text_username.layout.display = ""

## Dashboard UI

### Dashboard View

In [59]:
class DashboardView(ViewBase):

    title = make_title("Dashboard")
    title.isIgnored = True

    btn_sign_out = widgets.Button(
        description="Sign Out",
        disabled=False,
        button_style="danger"
    )
    btn_sign_out.isIgnored = True

    header = widgets.HBox([title, btn_sign_out], layout=widgets.Layout(
        justify_content="space-between",
        align_items="center",
        width="auto",
    ))
    header.isIgnored = True

    welcome_msg = widgets.HTML()
    welcome_msg.isIgnored = True

    options = widgets.Select(
        options=["Quadratic Equation"],
        description="Topics: ",
        disabled=False,
        style={
            'description_width': 'initial'
        }
    )
    options.isIgnored = True

    btn_proceed = widgets.Button(
        description="Proceed",
        disabled=False,
        button_style="info"
    )
    btn_proceed.isIgnored = True

    container_options = widgets.VBox([options, btn_proceed], layout=widgets.Layout(
        align_items="center",
        width="30em"
    ))
    container_options.isIgnored = True

    center = widgets.HBox([welcome_msg, container_options], layout=widgets.Layout(
        margin="1px 0",
        width="auto",
        display="grid"
    ))
    center.isIgnored = True

    Layout = widgets.AppLayout(
        header=header,
        left_sidebar=None,
        center=center,
        right_sidebar=None,
        footer=None,
        layout=widgets.Layout(
            width="100%",
            padding="1em",
        )
    )

    def inject_before_init_sync(self):
        self.binder(self.welcome_msg, AppState.name.name, lambda x: f"<strong>Welcome back! How are you, <span style='color:green'>{x}</span>?<strong/>")

### Dashboard Controller

In [60]:
class DashboardController(ControllerBase):
    _obj_view = DashboardView

    def on_btn_sign_out(self, event):
        self.router.go(MainMenuController)

    def on_btn_proceed(self, event):
        match self.view.options.value:
            case "Quadratic Equation":
                self.router.go(QuadraticEquationController, isRebuild = True)

## Single Question UI (Helper Classes)

### Single Question View

In [61]:
class SingleQuestionGeneratorView(ViewBase):
    prompt = widgets.HTML()
    prompt.isIgnored = True

    answer_input = widgets.Text(
        placeholder="Enter your answer",
        disabled=False
    )
    answer_input.isIgnored = True

    correct = widgets.HTML("<strong style='color:green'>Correct!</strong>",
                           layout=widgets.Layout(
                               display="none"
    ))
    correct.isIgnored = True

    incorrect = widgets.HTML("<strong style='color:red'>Incorrect! Try again or show the solution</strong>",
                             layout=widgets.Layout(
                                display="none"
    ))
    incorrect.isIgnored = True

    solution = widgets.HTML(
        layout = widgets.Layout(
            display="none"
    ))
    solution.isIgnored = True

    show_solution_btn = widgets.Button(
        description="Show Solution",
        disabled=False,
        button_style="warning",
        layout=widgets.Layout(display="none")
    )
    show_solution_btn.isIgnored = True

    submit_btn = widgets.Button(
                description="Submit",
                disabled=False,
                button_style="info"
    )
    submit_btn.isIgnored = True

    btn_container = widgets.HBox([submit_btn, show_solution_btn], layout = widgets.Layout(width="85%", justify_content = "space-around"))
    btn_container.isIgnored = True

    quiz_container = widgets.VBox([prompt, answer_input, solution, btn_container, correct, incorrect], layout=widgets.Layout(
        border="3px solid",
        padding="1em",
        margin="3em 0",
        align_items="center",
        justify_content="center",
        width="50em",
        height="20em",
        display="flex"
    ))

### Single Question Controller

In [62]:
class SingleQuestionGeneratorController(ControllerBase):
    _obj_view = SingleQuestionGeneratorView
    _required_arguments = {
        "question_generator" : Quiz,
        "title" : str,
    }

    async def inject_before_show_async(self):
        self.isSubmitted = False
        self.question = await self.get_unseen_question()
        self.view.prompt.value = f"<strong>{self.question.question}</strong>"
        _sol = self.question.solution.replace("\n", "<br>")
        self.view.solution.value = f"<strong>{_sol}</strong>"

    async def get_unseen_question(self):
        seen_question_ids = list(x["questionId"]
                                 for x in await AttemptModel.find(
            AttemptModel.userId == self.appstate.userId
        ).aggregate([{"$project" : {"questionId": 1, "_id": 0}}]).to_list())

        unseen_question = await QuestionModel.find(
            {"$and": [{"title": self.title}, {"_id": {"$nin": seen_question_ids}}]}
        ).aggregate([{"$sample": {"size" : 1}}]).to_list()

        if len(unseen_question) < 1:
            while True:
                try:
                    question_content = self.question_generator()

                    new_question_model = QuestionModel(
                        question_content = question_content,
                        title = self.title,
                        question_hash = hashlib.sha256(question_content.question.encode()).hexdigest()
                    )
                    await new_question_model.insert()
                    unseen_question.append(new_question_model)
                    break
                except DuplicateKeyError:
                    pass # Try again

            self.attempt = AttemptModel(
                userId = self.appstate.userId,
                questionId = unseen_question[0].id,
                isCorrect = False
            )
            await self.attempt.insert()
            return unseen_question[0].question_content

        else:
            self.attempt = AttemptModel(
                userId = self.appstate.userId,
                questionId = unseen_question[0]["_id"],
                isCorrect = False
            )
            await self.attempt.insert()
            return Quiz(**unseen_question[0]["question_content"])

    def on_show_solution_btn(self, event):
        self.view.solution.layout.display = ""

    async def on_submit_btn(self, event):
        isCorrect = any(self.view.answer_input.value.strip() == x for x in self.question.answer)
        self.view.correct.layout.display="none"
        self.view.incorrect.layout.display="none"
        if isCorrect:
            self.view.correct.layout.display=""
        else:
            self.view.incorrect.layout.display=""
            self.view.show_solution_btn.layout.display=""

        prev = self.attempt.isCorrect
        self.attempt.isCorrect = isCorrect or self.attempt.isCorrect
        self.attempt.timesOfAnswering += 1 if not prev else 0
        await self.attempt.save()

        self.isSubmitted = True

## Quadratic Equation UI

#### Quadratic Equation View

In [63]:
class QuadraticEquationView(ViewBase):
    title = make_title("Quadratic Equation")

    btn_exit = widgets.Button(
        description = "Exit",
        button_style = "danger"
    )

#### Quadratic Equation Controller

In [64]:
class QuadraticEquationController(ControllerBase):
    _obj_view = QuadraticEquationView
    def inject_before_show_sync(self):
        self.view.to_render_widgets.extend(SingleQuestionGeneratorController(self.appstate, self.router, question_generator = self.gen, title="Quadratic Equation").clone(5))

    def on_btn_exit(self, event):
        self.router.go(DashboardController)

    def gen(self):
        #-------------------------------------------
        # Wrie your code here!
        #-------------------------------------------
        import random
        question = "Sample Question" + str(random.randint(0 ,1000))
        solution = "Sample Solution"
        answer = ["test1", "test2"]
        return Quiz(question = question, solution = solution, answer = answer)

## Entry Point

In [65]:
print("Trying to connect to the database...")

# Create a new client and connect to the server
client = AsyncIOMotorClient(userdata.get("MongoDBAtlasConnectionString")) # The connection string will be revoked after WA3 is graded

await init_beanie(database=client["WA3"], document_models=[UserModel, QuestionModel, AttemptModel])
await client.admin.command('ping')

print("Connected!")
time.sleep(0.3)

clear_output()

router = Router()
router.register_one(MainMenuController)
router.register_one(RegisterController)
router.register_one(LoginController)
router.register_one(DashboardController)
router.register_one(QuadraticEquationController)

router.go(MainMenuController)

Output()